
# 🛡️ RACF Access Report Analysis

This notebook demonstrates how to parse a RACF-like mainframe access report stored in a fixed-width text format, extract user access details, identify unusual access configurations, and summarize the results for follow-up.

We'll be working with the file `sample_racf_data.txt`.
    


## 📦 Install Dependencies

If you haven't already installed `pandas`, run:

```bash
pip install pandas
```
    

In [1]:
import pandas as pd
import re

## 📂 Load RACF Report

In [3]:
with open("sample_racf_data.txt", "r") as file:
    lines = file.readlines()

# Preview first 20 lines
for line in lines[:20]:
    print(line.strip())

LISTGRP *
INFORMATION FOR GROUP PAYROLLB
SUPERIOR GROUP=RESEARCH     OWNER=IBMUSER   CREATED=06.123
NO INSTALLATION DATA
NO MODEL DATA SET
TERMUACC
NO SUBGROUPS
USER(S)=      ACCESS=      ACCESS COUNT=      UNIVERSAL ACCESS=
IBMUSER       JOIN          000000               ALTER
CONNECT ATTRIBUTES=NONE
REVOKE DATE=NONE                  RESUME DATE=NONE
DAF0          CREATE        000000               READ
CONNECT ATTRIBUTES=NONE
REVOKE DATE=NONE                  RESUME DATE=NONE
IA0           CREATE        000000               READ
CONNECT ATTRIBUTES=ADSP SPECIAL OPERATIONS
REVOKE DATE=NONE                  RESUME DATE=NONE
AEH0          CREATE        000000               READ
CONNECT ATTRIBUTES=NONE
REVOKE DATE=NONE                  RESUME DATE=NONE


## 📝 Parse User Access Records

In [4]:
# Initialize lists to hold parsed records
records = []
current_group = ""

for i, line in enumerate(lines):
    if "INFORMATION FOR GROUP" in line:
        current_group = line.strip().split()[-1]

    # Identify user lines: starts with a non-empty, non-space string followed by access keywords
    match = re.match(r"^\s*(\S+)\s+(JOIN|CREATE|CONNECT|USE)\s+(\d{6})\s+(\S+)", line)
    if match:
        user, access, access_count, universal_access = match.groups()

        # Look ahead for CONNECT ATTRIBUTES line
        attr_line = lines[i + 1].strip() if (i + 1) < len(lines) else ""
        attr_match = re.search(r"CONNECT ATTRIBUTES=(.*)", attr_line)
        attributes = attr_match.group(1) if attr_match else "NONE"

        records.append(
            {
                "Group": current_group,
                "User": user,
                "Access": access,
                "Access Count": int(access_count),
                "Universal Access": universal_access,
                "Attributes": attributes,
            }
        )

# Convert to DataFrame
df = pd.DataFrame(records)
df.head()

,Group,User,Access,Access Count,Universal Access,Attributes
0,PAYROLLB,IBMUSER,JOIN,0,ALTER,NONE
1,PAYROLLB,DAF0,CREATE,0,READ,NONE
2,PAYROLLB,IA0,CREATE,0,READ,ADSP SPECIAL OPERATIONS
3,PAYROLLB,AEH0,CREATE,0,READ,NONE
4,RESEARCH,IBMUSER,JOIN,0,ALTER,NONE


## 📊 Analyze Access Data

In [5]:
# Count users by Access type
access_summary = df["Access"].value_counts()
print(access_summary)

# Identify users with ALTER access or SPECIAL OPERATIONS attribute
anomalies = df[
    (df["Universal Access"] == "ALTER")
    | (df["Attributes"].str.contains("SPECIAL OPERATIONS"))
]

anomalies

Access
CREATE     5
JOIN       4
USE        3
CONNECT    1
Name: count, dtype: int64


,Group,User,Access,Access Count,Universal Access,Attributes
0,PAYROLLB,IBMUSER,JOIN,0,ALTER,NONE
2,PAYROLLB,IA0,CREATE,0,READ,ADSP SPECIAL OPERATIONS
4,RESEARCH,IBMUSER,JOIN,0,ALTER,NONE
6,RESEARCH,IA0,CONNECT,4,READ,ADSP SPECIAL OPERATIONS


## 📑 Prepare Follow-Up Report

In [6]:
# Create a concise follow-up report
follow_up = anomalies[
    ["Group", "User", "Access", "Universal Access", "Attributes"]
].copy()
follow_up["Notes"] = "Review access appropriateness with system owner"

follow_up.reset_index(drop=True, inplace=True)
follow_up

,Group,User,Access,Universal Access,Attributes,Notes
0,PAYROLLB,IBMUSER,JOIN,ALTER,NONE,Review access appropriateness with system owner
1,PAYROLLB,IA0,CREATE,READ,ADSP SPECIAL OPERATIONS,Review access appropriateness with system owner
2,RESEARCH,IBMUSER,JOIN,ALTER,NONE,Review access appropriateness with system owner
3,RESEARCH,IA0,CONNECT,READ,ADSP SPECIAL OPERATIONS,Review access appropriateness with system owner



## 🎉 Summary

In this notebook, we:
- Parsed a RACF-like access report from a fixed-width text file
- Extracted key fields into a structured DataFrame
- Analyzed access configurations for high-risk permissions
- Summarized anomalies requiring follow-up with system owners

🖥️ Use this as a starting point for mainframe audit automation projects!
    